<a href="https://colab.research.google.com/github/Bibirara/Python-learning-stuff/blob/main/Markov_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install quantecon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 3.4 MB/s eta 0:00:00


In [10]:
import matplotlib.pyplot as plt
import quantecon as qe
import numpy as np
import networkx as nx
from matplotlib import cm
import matplotlib as mpl
P = np.array([[0.68, 0.12, 0.20],
              [0.50, 0.24, 0.26],
              [0.36, 0.18, 0.46]])
P_power = np.linalg.matrix_power(P, 2)
P_power

array([[0.5944, 0.1464, 0.2592],
       [0.5536, 0.1644, 0.282 ],
       [0.5004, 0.1692, 0.3304]])

In [13]:
ψ_star_p = P_power[2]
ψ_star_p

array([0.5004, 0.1692, 0.3304])

In [14]:
ts_length = 10
num_distributions = 25
plot_distribution(P, ts_length, num_distributions)

NameError: name 'plot_distribution' is not defined